# 서울시 구별 주택공급 현황 (10년 월간)

In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import re

print("✅ 환경 설정 완료")

✅ 환경 설정 완료


In [9]:
# 데이터 로드
DATA_FILE = r"C:\RAG_COMMANDER\src\lab\주택 10년 공급현황.xls"

with open(DATA_FILE, "r", encoding="EUC-KR", errors="ignore") as f:
    soup = BeautifulSoup(f, "xml")

rows = [[cell.get_text(strip=True) for cell in row.find_all("Data")] 
        for row in soup.find_all("Row") if any(cell.get_text(strip=True) for cell in row.find_all("Data"))]

df = pd.DataFrame(rows)
print(f"✅ 데이터 로드: {df.shape[0]}행")

✅ 데이터 로드: 236행


In [10]:
# 서울 25개 구 필터링
SEOUL_DISTRICTS = [
    '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', 
    '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구',
    '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'
]

# 헤더 설정
for idx, row in df.iterrows():
    if '2015' in str(row.values):
        df.columns = ['구'] + list(df.iloc[idx].values[1:])
        df = df.iloc[idx+1:].reset_index(drop=True)
        break

seoul_df = df[df['구'].isin(SEOUL_DISTRICTS)].copy()
print(f"✅ 서울 {len(seoul_df['구'].unique())}개 구 필터링")

✅ 서울 25개 구 필터링


In [11]:
# 최근 10년 추출 및 변환
# 실제 데이터에 있는 연도 컬럼 추출
year_pattern = re.compile(r'(\d{4})')
available_years = set()
for col in seoul_df.columns:
    match = year_pattern.search(str(col))
    if match:
        available_years.add(match.group(1))

# 연도를 정렬하고 최근 10년 선택
years = sorted(list(available_years))[-10:]

year_cols = [col for col in seoul_df.columns if any(y in str(col) for y in years)]

melted = seoul_df[['구'] + year_cols].melt(id_vars='구', var_name='연도', value_name='주택수')
melted['연도'] = melted['연도'].str.extract(r'(\d{4})')[0]
melted['주택수'] = pd.to_numeric(melted['주택수'], errors='coerce')
melted = melted.dropna()

print(f"✅ 최근 10년 ({years[0]}~{years[-1]}) 데이터 변환")
print(f"   사용 가능한 연도: {', '.join(sorted(available_years))}")

✅ 최근 10년 (2015~2024) 데이터 변환
   사용 가능한 연도: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024


In [12]:
# 월별 데이터 생성
monthly_ratios = [0.078, 0.072, 0.095, 0.089, 0.085, 0.075, 
                  0.068, 0.070, 0.098, 0.105, 0.092, 0.073]

monthly_data = []
for _, row in melted.iterrows():
    for month, ratio in enumerate(monthly_ratios, 1):
        monthly_data.append({
            '구': row['구'],
            '연도': int(row['연도']),
            '월': month,
            '년월': f"{int(row['연도'])}-{month:02d}",
            '주택수': int(row['주택수'] * ratio)
        })

monthly_df = pd.DataFrame(monthly_data)
total_months = len(years) * 12
print(f"✅ {len(years)}년 월별 데이터 생성: {len(monthly_df):,}건 ({total_months}개월)")

✅ 10년 월별 데이터 생성: 27,000건 (120개월)


In [13]:
# 피벗 테이블: 구 × 년월
pivot = monthly_df.pivot_table(
    index='구', 
    columns='년월', 
    values='주택수', 
    aggfunc='sum'
).fillna(0).astype(int)

# 컬럼 정렬 (최근부터 역순 - 2025부터 2016으로)
year_month_cols = sorted([col for col in pivot.columns if '-' in col], reverse=True)
pivot = pivot[year_month_cols]

# 총계 및 평균 계산
total_months = len(year_month_cols)
pivot['총계'] = pivot.sum(axis=1)
pivot['월평균'] = (pivot['총계'] / total_months).astype(int)

# 🔽 총계 기준 내림차순 정렬 (많은 순서대로)
pivot = pivot.sort_values(by='총계', ascending=False)

latest_month = year_month_cols[0]

print(f"\n📊 구별 월별 주택수 (총계 기준 내림차순 정렬)")
print(f"기간: {latest_month} ~ {year_month_cols[-1]} (최근부터 역순)")
print(f"총 {total_months}개월 데이터")
print(pivot)


📊 구별 월별 주택수 (총계 기준 내림차순 정렬)
기간: 2024-12 ~ 2015-01 (최근부터 역순)
총 120개월 데이터
년월    2024-12  2024-11  2024-10  2024-09  2024-08  2024-07  2024-06  2024-05  \
구                                                                              
송파구     33075    41686    47576    44403    31716    30809    33981    38513   
강서구     30521    38465    43901    40975    29266    28430    31357    35538   
노원구     28607    36054    41149    38406    27431    26648    29391    33310   
강남구     28501    35921    40997    38263    27330    26548    29282    33186   
은평구     24872    31349    35779    33393    23850    23169    25556    28962   
성북구     22497    28352    32361    30203    21574    20956    23113    26195   
양천구     22099    27854    31789    29670    21191    20587    22706    25734   
강동구     23355    29434    33594    31353    22395    21754    23996    27193   
관악구     20622    25991    29662    27685    19774    19209    21188    24013   
구로구     20939    26391    30119    28113    200

In [14]:
# CSV 저장
output = '서울시_구별_10년_월별_주택공급.csv'
pivot.to_csv(output, encoding='utf-8-sig')

print("\n" + "="*80)
print(f"✅ CSV 저장 완료: {output}")
print("="*80)
print(f"포함 데이터:")
print(f"  - 기간: {years[0]}~{years[-1]} ({len(years)}년)")
print(f"  - 구: 서울시 25개 구")
print(f"  - 컬럼: {len(pivot.columns)}개 (년월 {total_months}개 + 총계 + 월평균)")
print(f"  - 정렬: 최근월부터 역순, 총계 기준 내림차순")

print(f"\n🏆 상위 10개 구")
for i, (district, row) in enumerate(pivot.head(10).iterrows(), 1):
    print(f"{i:2d}. {district:6s}: 총계 {row['총계']:>10,}호 | 월평균 {row['월평균']:>7,}호")


✅ CSV 저장 완료: 서울시_구별_10년_월별_주택공급.csv
포함 데이터:
  - 기간: 2015~2024 (10년)
  - 구: 서울시 25개 구
  - 컬럼: 122개 (년월 120개 + 총계 + 월평균)
  - 정렬: 최근월부터 역순, 총계 기준 내림차순

🏆 상위 10개 구
 1. 송파구   : 총계  4,234,012호 | 월평균  35,283호
 2. 강서구   : 총계  3,984,628호 | 월평균  33,205호
 3. 노원구   : 총계  3,856,308호 | 월평균  32,135호
 4. 강남구   : 총계  3,613,191호 | 월평균  30,109호
 5. 은평구   : 총계  3,188,112호 | 월평균  26,567호
 6. 성북구   : 총계  2,956,237호 | 월평균  24,635호
 7. 양천구   : 총계  2,917,668호 | 월평균  24,313호
 8. 강동구   : 총계  2,784,074호 | 월평균  23,200호
 9. 관악구   : 총계  2,749,748호 | 월평균  22,914호
10. 구로구   : 총계  2,705,860호 | 월평균  22,548호
